In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os
import numpy as np

### RESNET 32

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36
!pip install nvidia-ml-py3
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19189 sha256=8fff211b23d25d6ca08f0cae7f7650d135f847eda2e7e3596c674541015289fe
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7f/26/a3/33f2079871e2bebb3f53a2b21c3ec64129b8efdd18a6263a52
Successfully built nvidia-ml-py3
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
  Cloning https://github.com/NVIDIA/apex.git to /tmp/pip-install-a_s2xp9d/apex
  Running command git clone -q https://github.com/NVIDIA/apex.git /tmp/pip-install-a_s2xp9d/apex
  Running command git submodule update --init --recursive -q
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200704-py3-none-any.whl size=41891 sha256=63b740aae210dd181690490f413e0905461f06218992787a9cc09fe1e40f49db
  Stored in directory: /home/ec2-user/.cache/pip/wheels/fc/e7/9a/a35290ff6bb619b7c14477a31639990d58544bef99de6f7227
  Created wheel for apex: filename=ape

In [4]:
# Need to add this to requirements.txt
!pip install tensorboard

     |████████████████████████████████| 3.0 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 20.0 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 49.1 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 59.5 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 54.4 MB/s eta 0:00:01
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=864b6906079f638a4cdfd96b1ef93422520127eb00c20ea2956102a40b5d08e6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/af/84/3962a6af7b4ab336e951b7877dcfb758cf94548bb1771e0679
Successfully built absl-py
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [67]:
#Train the model per the settings specified in the original RESNET paper
# os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
# !python train.py --config configs/cifar/resnet.yaml \
#     model.resnet.depth 32 \
#     train.batch_size 128 \
#     dataset.name CIFAR10 \
#     train.base_lr 0.1 \
#     train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00 \
#     scheduler.epochs 1
pass

In [5]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 32 \
    train.batch_size 128 \
    dataset.name CIFAR10_RA_1_20 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00 \
    scheduler.epochs 400

[2020-07-10 20:06:30] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_1_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 32
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
 

In [6]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
#!python train.py --config configs/cifar/resnet.yaml \
!python train.py --config /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR


[2020-07-11 00:00:29] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 32
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_typ

In [7]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/test_results_0400_cifar10

[2020-07-11 15:30:00] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 26.90it/s]
[2020-07-11 15:30:04] __main__ INFO: Elapsed 2.94
[2020-07-11 15:30:04] __main__ INFO: Loss 1.1550 Accuracy 0.8226


In [8]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/test_results_0050_cifar10

[2020-07-11 15:31:05] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 27.10it/s]
[2020-07-11 15:31:09] __main__ INFO: Elapsed 2.92
[2020-07-11 15:31:09] __main__ INFO: Loss 0.4081 Accuracy 0.8897


In [9]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset - without training on unaugmented data
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/test_results_0400_cifar101

[2020-07-11 15:39:38] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.91it/s]
[2020-07-11 15:39:39] __main__ INFO: Elapsed 0.90
[2020-07-11 15:39:39] __main__ INFO: Loss 1.9384 Accuracy 0.6975


In [31]:
#npzf = np.load('resnet_basic_32_ra_2_20/predictions_400_CIFAR101.npz')
npzf = np.load('/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00/test_results_0400_cifar101/predictions.npz')
#experiments/resnet_basic_32_ra_2_20/exp00/test_results_0400_cifar101/predictions.npz
print(len(npzf['labels']))

2000


In [14]:
!pwd

/home/ec2-user/SageMaker/w210-capstone/aws-notebooks


In [10]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/test_results_0050_cifar101

[2020-07-11 15:40:27] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 18.11it/s]
[2020-07-11 15:40:28] __main__ INFO: Elapsed 0.89
[2020-07-11 15:40:28] __main__ INFO: Loss 0.7781 Accuracy 0.8005


In [11]:
import pandas as pd
a = pd.Series(['resnet_basic_32_ra_1_20', 400, 'cifar10', 1.1550, 0.8226]) #Loss 1.1550 Accuracy 0.8226
b = pd.Series(['resnet_basic_32_ra_1_20', 400, 'cifar10.1', 1.9384, 0.6975]) #Loss 1.9384 Accuracy 0.6975

c = pd.Series(['resnet_basic_32_ra_1_20_refined400', 50, 'cifar10', 0.4081, 0.8897]) #Loss 0.4081 Accuracy 0.8897
d = pd.Series(['resnet_basic_32_ra_1_20_refined400', 50, 'cifar10.1', 0.7781, 0.8005]) #0.7781 Accuracy 0.8005
               
df_results = pd.concat([a,b,c,d], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 92.5 if row[2] == 'cifar10' else 84.9), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (92.0, 93.0) if row[2] == 'cifar10' else (83.2, 86.4)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_32_ra_1_20,400,cifar10,1.155,0.8226,92.5,"(92.0, 93.0)"
1,resnet_basic_32_ra_1_20,400,cifar10.1,1.9384,0.6975,84.9,"(83.2, 86.4)"
2,resnet_basic_32_ra_1_20_refined400,50,cifar10,0.4081,0.8897,92.5,"(92.0, 93.0)"
3,resnet_basic_32_ra_1_20_refined400,50,cifar10.1,0.7781,0.8005,84.9,"(83.2, 86.4)"


In [35]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0400_cifar101/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0400_cifar101/predictions.npz'

In [16]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/resnet_basic_32_ra_1_20'
path = '/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_1_20'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)